In [6]:
!python /app/app/backend_pre_start.py

# Run migrations
!cd .. && alembic upgrade head

# Create initial data in DB
# python /app/app/initial_data.py
!python ../initial_brokers.py

Total errors are:
		set()
INFO  [alembic.runtime.migration] Context impl PostgresqlImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> b7b29fde792a, first revision


In [1]:
import time
import json
from pedesis.shortcuts import get_settings, get_engine
from pedesis.station.controller import Station

In [2]:
station = Station(get_settings())
station.start_all_public_components()

2023-06-16 08:25:33 [info     ] Station station --start-- public components.
2023-06-16 08:25:33 [info     ] Station station --start-- setup databases component.
2023-06-16 08:25:33 [info     ] Station station --start-- setup controller='pedesis.db.controller.LiveDataBase' name='default' username='postgres' password='postgres' host='db' port=5432 dialect='postgresql' driver='psycopg2' database='postgres' url=None database.
2023-06-16 08:25:33 [info     ] Station station --end-- setup controller='pedesis.db.controller.LiveDataBase' name='default' username='postgres' password='postgres' host='db' port=5432 dialect='postgresql' driver='psycopg2' database='postgres' url='postgresql+psycopg2://postgres:postgres@db:5432/postgres' database.
2023-06-16 08:25:33 [info     ] Station station --end-- setup start_databases component.
2023-06-16 08:25:33 [info     ] Station station --start-- setup caches component.
2023-06-16 08:25:33 [info     ] Station station --start-- setup controller='pedesis.c

In [3]:
engines = station._engines_name
engines

['multi_long_triple_ema_scalp',
 'scalp_short_macd',
 'swing_long_bollinger_bands']

In [6]:
macd = get_engine(engines[0])
macd

In [5]:
base_srls_configs = dict(
    calculators=macd._settings.installed_srls,
    timeframes=macd._settings.srl_timeframes,
    engine=macd._settings.engine_property
)
base_srls_configs

{'calculators': ['pedesis.components.srl_calculator.calculators.williams_fractal'],
 'timeframes': ['5m',
  '15m',
  '30m',
  '1h',
  '2h',
  '4h',
  '6h',
  '12h',
  '1d',
  '3d',
  '1w',
  '1M'],
 'engine': EngineProperty(name='scalp_short_macd', mode=<EngineMode.Scalp: 'scalp'>, native_broker='pedesis.components.broker.templates.okx', market='future')}

In [6]:
symbols = macd._get_symbols_router_list()
symbols

[app.scalp_short_macd.router.BTCUSDT,
 app.scalp_short_macd.router.ETHUSDT,
 app.scalp_short_macd.router.DOGEUSDT]

In [7]:
def make_srls_record(srls_configs):
    from pedesis.components.srl_calculator.models import SRLsModel, UniSymbol
    calcs = srls_configs['calculators']
    calcs.sort()
    tms = srls_configs['timeframes']
    tms.sort()
    uni_symbol_id = UniSymbol.get('id', symbol=srls_configs['symbol'].__name__)
    srls = SRLsModel(
        signal_engine_signature=srls_configs['engine'].srl_signature(),
        calculators=json.dumps(calcs),
        timeframes=json.dumps(tms),
        uni_symbol_id=uni_symbol_id
    ).save()
    srls_configs['engine'].add_srls_in_cache(uni_symbol_id, srls.id)
    return srls

In [8]:
btc_srls = base_srls_configs.copy()
btc_srls['symbol'] = symbols[0]
btc_srls

{'calculators': ['pedesis.components.srl_calculator.calculators.williams_fractal'],
 'timeframes': ['5m',
  '15m',
  '30m',
  '1h',
  '2h',
  '4h',
  '6h',
  '12h',
  '1d',
  '3d',
  '1w',
  '1M'],
 'engine': EngineProperty(name='scalp_short_macd', mode=<EngineMode.Scalp: 'scalp'>, native_broker='pedesis.components.broker.templates.okx', market='future'),
 'symbol': app.scalp_short_macd.router.BTCUSDT}

In [9]:
from pedesis.components.srl_calculator.maker import SRLsStarter
btc_starter = SRLsStarter(make_srls_record(btc_srls).get_configs())
btc_starter

In [10]:
def srls_start(srls_starter):
    from pedesis.components.srl_calculator.maker import get_srl, SRLConfigs
    from pedesis.logger import get_logger
    from pedesis.cache import get_redlock
    import itertools
    
    _logger = get_logger()
    _logger.info(f"SRLsStarter --start-- make SRL class for this SRLs: {srls_starter.configs.cache_uid}")
    srl_tasks = []
    for engine, uni_symbol_id, tm, calc in itertools.product([srls_starter.configs.engine], [srls_starter.configs.uni_symbol_id], srls_starter.configs.timeframes, srls_starter.configs.calculators):
        conf = SRLConfigs(
            engine=engine,
            uni_symbol_id=uni_symbol_id,
            timeframe=tm,
            calculator=calc,
        )
        _logger.info(f"SRLsStarter --start-- create SRL for {conf.cache_uid}.")
        with get_redlock(conf.get_redlock_key()):
            srl = get_srl(conf, log=_logger)
            srl_tasks.append(srl.make_task())
            _logger.info(f"SRLsStarter --end-- create SRL for {conf.cache_uid}.")
            srl.add_srls(srls_starter.configs.dbid)
        srls_starter.installed_srl[srl.configs.dbid] = srl._calculator_settings

In [11]:
srls_start(btc_starter)

/usr/local/lib/python3.10/site-packages/pedesis/components/srl_calculator/controller.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srline_ohlcv.loc[srline_ohlcv.index, 'index_price'] = srline_ohlcv.get(cols).mean(axis=1)
/usr/local/lib/python3.10/site-packages/pedesis/components/srl_calculator/controller.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srline_ohlcv.loc[srline_ohlcv.index, 'index_price'] = srline_ohlcv.get(cols).mean(axis=1)
/usr/local/lib/python3.10/site-packages/pedesis/compon

In [12]:
btc_starter.installed_srl

{1: CalculatorTimeFrameSettings(timeframe='12h', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 10}),
 2: CalculatorTimeFrameSettings(timeframe='15m', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 20}),
 3: CalculatorTimeFrameSettings(timeframe='1M', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 2}),
 4: CalculatorTimeFrameSettings(timeframe='1d', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, other=None), first_std_srl=0.025, other={'period': 7}),
 5: CalculatorTimeFrameSettings(timeframe='1h', srlevel_std_multiplier=1.05, data_settings=DataSettings(start_time=1609459200000.0, required_length=None, ot

In [13]:
from pedesis.components.srl_calculator.models import *

SRLineRaw.all(in_df=True)

,open,id,srl_id,timestamp,index_price,high,low,close,volume
0,41407.6,1,1,1.610107e+12,40657.125,42077.6,38596.0,40547.3,3577496.0
1,34221.1,2,1,1.610366e+12,33903.600,35601.7,30431.4,35360.2,7602135.0
2,37040.6,3,1,1.611922e+12,35840.950,38600.0,33500.0,34223.2,4271187.0
3,30651.2,4,1,1.611749e+12,30489.750,31675.2,29258.4,30374.2,3712610.0
4,30864.2,5,1,1.611274e+12,30880.350,32060.0,28860.0,31737.2,4537623.0
...,...,...,...,...,...,...,...,...,...
15910,20961.7,15911,12,1.655813e+12,21201.625,21749.2,20851.0,21244.6,5937205.0
15911,16211.7,15912,12,1.669637e+12,16204.550,16399.0,16000.0,16207.5,3924396.0
15912,16498.7,15913,12,1.672402e+12,16488.275,16581.0,16337.0,16536.4,1963644.0
15913,24576.0,15914,12,1.676549e+12,24751.250,25251.2,24290.3,24887.5,8384968.0


In [14]:
srlines_raw_stmt = SRLineRaw.get_select().where(SRLineRaw.srl_id.in_(list(btc_starter.installed_srl.keys())))
srlines_raw = SRLineRaw.query(srlines_raw_stmt, in_df=True)
srline_stmt = SRLine.get_select().where(SRLine.srline_raw_id.in_(list(srlines_raw.id)))
srlines = SRLine.query(srline_stmt, in_df=True)

In [15]:
srlines

,id,srls_id,srline_raw_id,srlevel_id


In [16]:
srlines_raw

,open,id,srl_id,timestamp,index_price,high,low,close,volume
0,41407.6,1,1,1.610107e+12,40657.125,42077.6,38596.0,40547.3,3577496.0
1,34221.1,2,1,1.610366e+12,33903.600,35601.7,30431.4,35360.2,7602135.0
2,37040.6,3,1,1.611922e+12,35840.950,38600.0,33500.0,34223.2,4271187.0
3,30651.2,4,1,1.611749e+12,30489.750,31675.2,29258.4,30374.2,3712610.0
4,30864.2,5,1,1.611274e+12,30880.350,32060.0,28860.0,31737.2,4537623.0
...,...,...,...,...,...,...,...,...,...
15910,20961.7,15911,12,1.655813e+12,21201.625,21749.2,20851.0,21244.6,5937205.0
15911,16211.7,15912,12,1.669637e+12,16204.550,16399.0,16000.0,16207.5,3924396.0
15912,16498.7,15913,12,1.672402e+12,16488.275,16581.0,16337.0,16536.4,1963644.0
15913,24576.0,15914,12,1.676549e+12,24751.250,25251.2,24290.3,24887.5,8384968.0


In [17]:
must_add = set(srlines_raw.id).difference(set(srlines.srline_raw_id))

In [30]:
self = btc_starter
dbid = self.configs.dbid
symbol = UniSymbol.get('symbol', id=self.configs.uni_symbol_id)

srline_df = SRLine.query(SRLine.get_select(srls_id=dbid), in_df=True)
base_srlines_raw = srlines_raw[srlines_raw['id'].isin(must_add)]
base_srlines_raw.sort_values('timestamp', ascending=True, inplace=True)
srlevels_dfs = {
    srl_id: SRLevel.get_raw_srlevel_df(
    srls_id=self.configs.dbid,
    band_mult=self.installed_srl[srl_id].srlevel_std_multiplier
) for srl_id in self.installed_srl
}
srlevels_exist_ids = {
    i[0]: list(i[1].id.values) for i in srlevels_dfs.items()
}

In [19]:
srlevels_exist_ids

{1: [],
 2: [],
 3: [],
 4: [],
 5: [],
 6: [],
 7: [],
 8: [],
 9: [],
 10: [],
 11: [],
 12: []}

In [28]:
!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 29.5 MB/s eta 0:00:0000:0100:01


In [33]:
import numba
from pedesis.components.srl_calculator.models import *

@numba.jit()
def add_levels(self, base_srlines_raw, srlevels_dfs, srline_df):
    for srline_raw in base_srlines_raw.itertuples(index=False):
        is_level_exist = True
        srlevel_df = srlevels_dfs[srline_raw.srl_id]
        srlevel = SRLevel.locate_srlevel_new(srlevel_df, srline_raw.index_price)
        if not isinstance(srlevel, pd.Series):
            is_level_exist = False
            srlevel_df = self.add_new_srlevel_to_df(
                srlevel_df=srlevel_df,
                srline_raw=srline_raw,
                srls_id=dbid
            )
            srlevels_dfs[srline_raw.srl_id] = srlevel_df
            srlevel = srlevel_df.iloc[-1]
        srlevel_id = srlevel.id
        srline_df = self.add_new_srline_to_df(
            srline_df=srline_df,
            srls_id=dbid,
            srline_raw_id=srline_raw.id,
            srlevel_id=srlevel_id
        )
        if is_level_exist:
            srlevel_df = self.update_srlevels_df(
                srlevel_df=srlevel_df,
                srlevel_id=srlevel_id,
                srline_raw=srline_raw,
                srline_df=srline_df,
                srline_raw_df=base_srlines_raw
            )
        srlevels_dfs[srline_raw.srl_id] = srlevel_df


/tmp/ipykernel_14/2792829677.py:4: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()


In [34]:
base_srlines_raw

,open,id,srl_id,timestamp,index_price,high,low,close,volume
13811,29308.6,13812,11,1.609474e+12,29352.400,29413.7,29308.6,29378.7,10214.0
11879,29225.3,11880,11,1.609476e+12,29182.650,29234.3,29082.5,29188.5,22532.0
2175,29197.2,2176,2,1.609484e+12,29051.850,29211.0,28841.9,28957.3,74447.0
8883,29047.9,8884,11,1.609485e+12,28974.525,29051.0,28841.9,28957.3,38035.0
5212,29454.9,5213,8,1.609504e+12,29365.575,29637.8,29141.6,29228.0,231468.0
...,...,...,...,...,...,...,...,...,...
13892,25999.8,13893,11,1.686631e+12,26002.275,26022.0,25976.2,26011.1,35739.0
12759,26099.8,12760,11,1.686638e+12,26111.075,26135.5,26098.1,26110.9,39308.0
7075,26170.0,7076,11,1.686646e+12,26218.525,26300.0,26169.9,26234.2,479828.0
13864,26240.9,13865,11,1.686659e+12,26288.650,26440.0,26173.8,26299.9,1411037.0


In [35]:
t1 = time.time()
add_levels(btc_starter, base_srlines_raw.iloc[:1000], srlevels_dfs, srline_df)
t2 = time.time()

/tmp/ipykernel_14/2792829677.py:4: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "add_levels" failed type inference due to: Untyped global name 'SRLevel': Cannot determine Numba type of <class 'sqlalchemy.orm.decl_api.DeclarativeMeta'>

File "../tmp/ipykernel_14/2792829677.py", line 9:
<source missing, REPL/exec in use?>

  @numba.jit()
/tmp/ipykernel_14/2792829677.py:4: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "add_levels" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "../tmp/ipykernel_14/2792829677.py", line 6:
<source missing, REPL/exec in use?>

  @numba.jit()
/usr/local/lib/python3.10/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "add_levels" was compiled in object mode without forceobj=True, but has lifted loops.

File "../tmp/ipykernel_14/2792829677.py", line 4:
<source mis

In [36]:
t2 - t1

32.19484543800354

In [39]:
t1 = time.time()
add_levels(btc_starter, base_srlines_raw.iloc[2000:3000], srlevels_dfs, srline_df)
t2 = time.time()

/usr/local/lib/python3.10/site-packages/pedesis/components/srl_calculator/models.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_srlevel_df.sort_values(by='quality', ascending=False, inplace=True)
/usr/local/lib/python3.10/site-packages/pedesis/components/srl_calculator/models.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_srlevel_df.sort_values(by='quality', ascending=False, inplace=True)
/usr/local/lib/python3.10/site-packages/pedesis/components/srl_calculator/models.py:286: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the do

In [40]:
t2 - t1

17.044589042663574

In [41]:
srlevels_dfs

{1:     id  srls_id  average_price    std_price  quality    upper_band  \
 0    1        1   40657.125000  1016.428125        1  41724.374531   
 1    2        1   33903.600000   847.590000        1  34793.569500   
 2    3        1   37639.950240  3014.969431      104  40805.668143   
 3    4        1   56762.737500   811.599486        2  57614.916960   
 4    5        1   44830.850000  1120.771250        1  46007.659813   
 5    6        1   51571.841667   298.105306        3  51884.852238   
 6    7        1   59922.262500   868.804424        2  60834.507146   
 7    8        1   56990.533333  2362.724040        3  59471.393575   
 8    9        1   64041.000000  1601.025000        1  65722.076250   
 9   10        1   49038.550000  1225.963750        1  50325.811937   
 10  11        1   35190.875000   879.771875        1  36114.635469   
 11  12        1   32816.150000   820.403750        1  33677.573938   
 12  13        1   31465.275000   786.631875        1  32291.238469   
 
 